<a href="https://www.kaggle.com/code/bilmthanh/final-version-coordinate?scriptVersionId=142948549" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
"""
    name: travelling-saleman-problem
    author: thanh.bl204606@sis.hust.edu.vn
    date: 2023-09-11 13:57:45.184949
"""

In [ ]:
pip install opencv-python

In [ ]:
#import thư viện
import cv2
import time
import math
import random
import datetime
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import IPython.display as display

In [ ]:
print(datetime.datetime.now())

In [ ]:
#path đến file dữ liệu và file lời giải tối ưu 
coordinate = [
    "/kaggle/input/coordinate/coordinate/29Y_TSP.txt", "/kaggle/input/coordinate/coordinate/48A_TSP.txt",
    "/kaggle/input/coordinate/coordinate/51eli_TSP.txt", "/kaggle/input/coordinate/coordinate/52berlin_TSP.txt",
    "/kaggle/input/coordinate/coordinate/70st_TSP.txt", "/kaggle/input/coordinate/coordinate/76pr_TSP.txt",
    "/kaggle/input/coordinate/coordinate/101eli_TSP.txt", "/kaggle/input/coordinate/coordinate/120gr_TSP.txt",
    "/kaggle/input/coordinate/coordinate/130ch_TSP.txt", "/kaggle/input/coordinate/coordinate/150ch_TSP.txt"
]
optimize = [
    "/kaggle/input/optimize1/optimization/29Y_TSP_opt.txt", "/kaggle/input/optimize1/optimization/48A_TSP_opt.txt",
    "/kaggle/input/optimize1/optimization/51eli_TSP_opt.txt", "/kaggle/input/optimize1/optimization/52berlin_TSP_opt.txt",
    "/kaggle/input/optimize1/optimization/70st_TSP_opt.txt", "/kaggle/input/optimize1/optimization/76pr_TSP_opt.txt",
    "/kaggle/input/optimize1/optimization/101eli_TSP_opt.txt", "/kaggle/input/optimize1/optimization/120gr_TSP_opt.txt",
    "/kaggle/input/optimize1/optimization/130ch_TSP_opt.txt", "/kaggle/input/optimize1/optimization/150ch_TSP_opt.txt"
]

In [ ]:
#block đọc dữ liệu file tọa độ thành phố và trả về ma trận khoảng cách
def readCoordinate(path, ftype='txt'):
    if ftype == 'txt':
        cities = []
        f = open(path)
        for i in f.readlines():
            node_city_val = i.split()
            cities.append([node_city_val[0], float(node_city_val[1]), float(node_city_val[2])])
    else:
        infile = open(path, 'r')
        cities = []
        for i, j in enumerate(infile.readlines()):
            node_city_val = j.split()
            if i > 6 and len(node_city_val) == 3 :
                cities.append([node_city_val[0], float(node_city_val[1]), float(node_city_val[2])])
        infile.close()
    dis_mat = []
    for cityA in cities:
        dis_vec = []
        for cityB in cities:
            dis = ((cityA[1] - cityB[1])**2 + (cityA[2] - cityB[2])**2)**0.5
            dis_vec.append(dis)
        dis_mat.append(dis_vec)
    return cities, dis_mat

#block code đọc đường đi tối ưu
def read_optimize_txt(file_path):
    try:
        with open(file_path, 'r') as file:           
            lines = file.readlines()
            numbers = [int(line.strip()) for line in lines]
            return numbers
    except FileNotFoundError:
        print(f"File '{file_path}' doesn,t exist.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [ ]:
#chọn file data;
index = 1
#đọc ra file tối ưu
optimal_road = read_optimize_txt(optimize[index])

#tính chi phí (cost) của file tối ưu
def finess(ls):
    cost = 0
    for i in range(1, len(ls)):
        cost += distance_matrix[ls[i] - 1][ls[i - 1] - 1]
    cost += distance_matrix[ls[0] - 1][ls[len(ls) - 1] - 1] 
    return cost

cities, distance_matrix = readCoordinate(coordinate[index])
print(f'FILE_NAME:        ...{coordinate[index][35:]}')
print(f'NUMBER_CITIES:    {len(distance_matrix)}')
print(f"OPTIMAL_SOLUTION: {finess(optimal_road)}")

<span style="font-size: 25px; bold: bolder">GENETIC ALGORITHM</span>


In [ ]:
def totalDistance(sol): #hàm tính độ thích nghi của lời giải
    
    moving_cost = 0
    
    for i in range(len(sol) - 1):
        citi1 = int(sol[i]) - 1   
        citi2 = int(sol[i + 1]) - 1
        moving_cost += distance_matrix[citi1][citi2]
        
    citi1 = int(sol[0]) - 1
    citi2 = int(sol[-1]) - 1
    moving_cost += distance_matrix[citi1][citi2]
    #tính chi phí quay lại thành phố ban đầu
    
    return moving_cost

In [ ]:
def createPopulation(cities, size):
    #tạo quần thể ban đầu;
    population = []
    
    for i in range(size):
        sol = list(np.array(cities.copy())[:,0])
        #các cá thể là random
        random.shuffle(sol)
        distance = totalDistance(sol)
        population.append([distance, sol])
        
    return population

In [ ]:
image_path = '/kaggle/input/picture/describe/GA/crossover.jpg'
image = Image.open(image_path)
resized_image = image.resize((700, 500))
display.display(resized_image)

In [ ]:
def multiPointCrossover(parent1, parent2, lenCities = len(distance_matrix)):
    #lai ghép giữa 2 cá thế trả về 2 cá thế mới
    points = sorted(random.sample(parent1, 2))
    point1 = int(points[0]) - 1
    point2 = int(points[1]) - 1
    #chọn ra 2 điểm lai ghép ngẫu nhiên
    
    newChild1 = parent2[point1: point2]
    c1 = 0
    c2 = 0
    for gen in parent1:
        if gen not in newChild1:
            c1 += 1
            if c1 <= point1:
                newChild1.append(gen)
            else:
                newChild1.insert(c2, gen)
                c2 += 1
                
    newChild2 = parent1[point1: point2]
    c1 = 0
    c2 = 0
    for gen in parent2:
        if gen not in newChild2:
            c1 += 1
            if c1 <= point1:
                newChild2.append(gen)
            else:
                newChild2.insert(c2, gen)
                c2 += 1
    return newChild1, newChild2

In [ ]:
image_path = '/kaggle/input/picture/describe/GA/mutation.jpg'
image = Image.open(image_path)
resized_image = image.resize((700, 500))
display.display(resized_image)

In [ ]:
def mutation(parent, lenCities = len(distance_matrix)):
    #đột biến 1 cá thể tạo ra một cá thế mới
    newChild = parent.copy()
    
    for _ in range(2):
        points = random.sample(newChild, 2)
        #chọn ra 2 điểm ngẫu nhiên
        temp = newChild[int(points[0]) - 1]
        newChild[int(points[0]) - 1] = newChild[int(points[1]) - 1]
        newChild [int(points[1]) - 1] = temp
        
    return newChild

In [ ]:
def selection(old_population, new_invivduals, population_size = 100):
    #tại mỗi thế hệ, sau khi lai ghép population_size / 2 lần, tức là 100 cá thế, lai ghép 50 lần
    #tạo ra 100 cá thể mới. Ở mỗi quần thế cũ và mới, chúng ta sẽ lấy 1 số lượng cá thể tốt nhất cố định
    #đảm bảo lời giải sau mỗi thế hệ đều được update. Và cũng không lấy quá nhiều để tạo sự đa dạng cho quần thể.
    new_population = []
    
    k1 = int(population_size / 50)
    k2 = int(population_size / 50)
    
    '''Keep k1 people from old generation'''
    tmp1 = sorted(old_population)
    for i in range(k1):
        new_population.append(tmp1[i])
        
    '''Keep k2 people from new generation'''
    tmp2 = sorted(new_invivduals)
    for i in range(k2):
        new_population.append(tmp2[i])
    '''Take random k3 people from the rest'''
    
    tmp3 = tmp1[k1:] + tmp2[k2:] #những cá thể còn lại
    k3 = population_size - k1 - k2
    new_population = new_population + random.choices(tmp3, k=k3)    
    return new_population

In [ ]:
def GA(
    population,
    lenCities = len(distance_matrix),
    population_size = 100,
    num_of_generations = 500,
    crossover_rate = 0.9,
    mutation_rate = 0.1,
    k = 4
):
    for _ in range(num_of_generations):
        
        new_inviduals = [] #lưu giữ các cá thể mới
        
        n = int(population_size/2) #số lần tạo cá thể mới
        
        for i in range(n):
            if random.random() <= crossover_rate: #lai ghép
                
                # chọn cá thể tốt nhất trong k cá thể ngẫu nhiên làm parent1
                father1 = sorted(random.choices(population, k=k))[0][1]
                # chọn cá thể tốt nhất trong k cá thể ngẫu nhiên làm parent2
                father2 = sorted(random.choices(population, k=k))[0][1]       
                # thực hiện lai ghép 2 cá thể này
                
                inv1, inv2 = multiPointCrossover(father1, father2, lenCities) #thực hiện lai ghép
                
                if random.random() <= mutation_rate: #mutation
                    inv1 = mutation(inv1, lenCities)
                    inv2 = mutation(inv2, lenCities)  
            else:
                inv1 = random.choices(population)[0][1]
                inv2 = random.choices(population)[0][1]          
            new_inviduals.append([totalDistance(inv1), inv1])
            new_inviduals.append([totalDistance(inv2), inv2]) 
            #cho các cá thể mới được tạo ra vào quần thể mới
            
        #lựa chọn các cá thể mới vào quần thể mới    
        population = selection(population, new_inviduals, population_size)  
    return sorted(population)

In [ ]:
minimum = 1e8
road = None
start = time.time()
seed_num = 30
for i in range(seed_num):
    print(f'Seed {i} is running..')
    random.seed(i)
    population = createPopulation(cities, size=100) # Khởi tạo quần thể đầu tiên
    pop = GA(population=population, lenCities=len(cities))
    if minimum > pop[0][0]:
        minimum = pop[0][0] #lưu kết quả tốt nhất
        road = pop[0][1] #lưu đoạn đường tốt nhất
print(f'RUN_TIME: {time.time() - start} sec')

In [ ]:
print(f'BEST_COST: {minimum}')

In [ ]:
print(f'BEST_SOLUTION: {road}')

<span style="font-size: 25px; bold: bolder">ANT COLONY OPTIMIZATION</span>


https://staff.washington.edu/paymana/swarm/stutzle99-eaecs.pdf

In [ ]:
#Ant Colony Optimization - Done
class Graph(object):
    def __init__(self, cost_matrix: list, rank: int):
        self.matrix = cost_matrix
        self.rank = rank
        self.pheromone = [[1 / (rank * rank) for j in range(rank)] for i in range(rank)]
        #khởi tạo pheromone ban đầu, pheromone ban đầu để những con kiến ban đầu không bị ảnh
        #hưởng quá nhiều bởi pheromone

In [ ]:
image_path = '/kaggle/input/picture/describe/ACO/update_pheromone.png'
image = Image.open(image_path)
resized_image = image.resize((1250, 300))
display.display(resized_image)

In [ ]:
class ACO(object):
    def __init__(self, ant_count: int, generations: int, alpha: float, beta: float, rho: float, q: int, strategy: int):
        self.Q = q                              #q:
        self.rho = rho                          #rho: tốc độ bay hơi của pheromone sau mỗi thế hệ
        self.beta = beta                        #beta:
        self.alpha = alpha                      #alpha:
        self.ant_count = ant_count              #ant_count: số lượng kiến ảo
        self.generations = generations          #số lượng thế hệ
        self.update_strategy = strategy         #strategy: chiến lược cập nhật pheromone
        
        
    def updatePheromone(self, graph: Graph, ants: list):  #update pheromone sau mỗi thế hệ kiếm ăn
        
        for i, row in enumerate(graph.pheromone):
            for j, col in enumerate(row):
                
                graph.pheromone[i][j] *= (1 - self.rho) #sau mỗi thế hệ, pheromone của mỗi cạnh sẽ giảm đi bằng 1 hệ s
                for ant in ants:
                    graph.pheromone[i][j] += ant.pheromone_each[i][j] # update pheromone thế hệ mới nhất
                    
                    
    def solve(self, graph: Graph):
        best_cost = 1e8
        best_solution = []
        
        for gen in range(self.generations):
            if gen % (self.generations / 20) == 0:
                print(f'Gen {gen} is running...')
                
            ants = [Ant(self, graph) for i in range(self.ant_count)]
            
            for ant in ants:
                for i in range(graph.rank - 1):
                    ant.select_next()
                ant.total_cost += graph.matrix[ant.visited[-1]][ant.visited[0]]
                #quay lại thành phố ban đầu
                if ant.total_cost < best_cost:
                    best_cost = ant.total_cost
                    best_solution = [] + ant.visited
                    
                # update pheromone for each ant;
                ant.update_pheromone_each()
                
            self.updatePheromone(graph, ants)
            #update pheromone local
        return best_solution, best_cost

In [ ]:
image_path = '/kaggle/input/picture/describe/ACO/form.png'
image = Image.open(image_path)
resized_image = image.resize((1350, 300))
display.display(resized_image)

In [ ]:
class Ant(object):
    
    def __init__(self, aco: ACO, graph: Graph):
        self.colony = aco      
        self.graph = graph
        self.total_cost = 0.0
        self.visited = []
        self.pheromone_each = []  # the local increase of pheromone
        self.possible = [i for i in range(graph.rank)]  # nodes which are allowed for the next selection
        self.eta = [[0 if i == j else 1 / graph.matrix[i][j] for j in range(graph.rank)] for i in range(graph.rank)]  # heuristic information
        start = random.randint(0, graph.rank - 1)  # start from any node
        self.visited.append(start)
        self.current = start
        self.possible.remove(start)
        
    def select_next(self): #chọn lựa bước đi tiếp theo cho từng kiến
        denominator = 0
        for i in self.possible: #chỉ xét những thành phố chưa đi qua
            
            denominator += self.graph.pheromone[self.current][i] ** self.colony.alpha * self.eta[self.current][i] ** self.colony.beta
            
        probabilities = [0 for i in range(self.graph.rank)] 
        
        for i in range(self.graph.rank):
            try:
                self.possible.index(i)  # test if allowed list contains i
                probabilities[i] = self.graph.pheromone[self.current][i] ** self.colony.alpha * self.eta[self.current][i] ** self.colony.beta / denominator
                #alpha và beta là các tham số thể hiện nên sự ảnh hưởng của pheromone hoặc thông tin heuristic vào sự lựa chọn của kiến
                
            except ValueError:
                pass  # do nothing
        selected = 0
        rand = random.random()
        for i, probability in enumerate(probabilities):
            rand -= probability
            if rand <= 0:
                selected = i
                break
        self.possible.remove(selected)                                 #loại bỏ thành phố khỏi những thành phố chưa đi 
        self.visited.append(selected)                                  #cho vào list visited
        self.total_cost += self.graph.matrix[self.current][selected]   #update total_cost
        self.current = selected                                        #update thành phố mà kiến đang đứng
                     
    def update_pheromone_each(self):
        
        self.pheromone_each = [[0 for j in range(self.graph.rank)] for i in range(self.graph.rank)]
        #xét ở toàn đồ thị
        
        for _ in range(1, len(self.visited)):
            #trong chuỗi
            i = self.visited[_ - 1]
            j = self.visited[_]
            #các chiến lược update pheromone;
            
            if self.colony.update_strategy == 1:  # ant-quality system: mật độ kiến
                self.pheromone_each[i][j] = self.colony.Q
            elif self.colony.update_strategy == 2:  # ant-density system: số lượng kiến
                self.pheromone_each[i][j] = self.colony.Q / self.graph.matrix[i][j]
            else:  # ant-cycle system
                self.pheromone_each[i][j] = self.colony.Q / self.total_cost

In [ ]:
def main():
    st = time.time()
    rank = len(distance_matrix)
    ants = 10
    gen = 100
    alpha = 1
    beta = 10
    rho = 0.5
    q = 1
    strategy = 3
    aco = ACO(ants, gen, alpha, beta, rho, q, strategy)
    graph = Graph(distance_matrix, rank)
    path, cost = aco.solve(graph)
    path.append(path[0])
    print(f'COST: {cost}')
    print(f'PATH: {path}')
    print(f'RUN_TIME: {time.time() - st} sec')
main()

<span style="font-size: 25px; bold: bolder">NEAREST NEIGHBOR</span>


In [ ]:
image_path = '/kaggle/input/picture/describe/NN/idea.jpg'
image = Image.open(image_path)
resized_image = image.resize((650, 550))
display.display(resized_image)

In [ ]:
def fitness(road): #hàm tính giá trị fitness;  
    cost = 0
    for i in range(1, len(road)):
        cost += distance_matrix[int(road[i])][int(road[i - 1])]
    return cost

In [ ]:
def nearest_neightbor(distance_matrix, road): #hàm tìm ra neighbor gần nhất tiếp theo;
    nearestCiti = -1
    minCiti_cost = 1e8
    ls = distance_matrix[road[len(road) - 1]]
    for i in range(len(ls)):
        if ls[i] < minCiti_cost and i not in road:
            nearestCiti = i
            minCiti_cost = ls[i]
    return nearestCiti

In [ ]:
best_cost = 1e8
bestRoad, bestRoot = None, None
for i in range(len(distance_matrix)): #vòng lặp
    road = []
    road.append(i) #set i là thành phố xuất phát
    while len(road) < len(distance_matrix): 
        road.append(nearest_neightbor(distance_matrix, road))
    road.append(i) #quay trở lại i
    if fitness(road) < best_cost:
        best_cost = fitness(road)
        bestRoad = road
        bestRoot = i

In [ ]:
print(f"BEST_SOLUTION: {best_cost}")
print(f'BEST SOL FROM ROOT: {bestRoot}')
print(f'ROAD: {bestRoad}')

<span style="font-size: 25px; bold: bolder">BRANCH AND BOUND</span>


In [ ]:
image_path = '/kaggle/input/picture/describe/BAB/idea.jpg'
image = Image.open(image_path)
resized_image = image.resize((700, 600))
display.display(resized_image)

In [ ]:
#Branch and bound to solve TSP
import math
maxsize = float('inf')

In [ ]:
def copyToFinal(curr_path):
    final_path[:N + 1] = curr_path[:]
    final_path[N] = curr_path[0]

In [ ]:
# Function to find the minimum edge cost
# having an end at the vertex i
def firstMin(adj, i):
    min = maxsize
    for k in range(N):
        if adj[i][k] < min and i != k:
            min = adj[i][k]
    return min

In [ ]:
# function to find the second minimum edge
# cost having an end at the vertex i
def secondMin(adj, i):
    first, second = maxsize, maxsize
    for j in range(N):
        if i == j:
            continue
        if adj[i][j] <= first:
            second = first
            first = adj[i][j]
        elif(adj[i][j] <= second and adj[i][j] != first):
            second = adj[i][j]
    return second

In [ ]:
time_limit = 10
def TSPRec(adj, curr_bound, curr_weight, level, curr_path, visited):
    global final_res
    if time.time() - start > time_limit:
        return
    if level == N:
        if adj[curr_path[level - 1]][curr_path[0]] != 0:
            curr_res = curr_weight + adj[curr_path[level - 1]][curr_path[0]]
            if curr_res < final_res:
                copyToFinal(curr_path)
                final_res = curr_res
                print(f'UPDATE_BEST_SOLUTION: {round(final_res, 2)} - TIME_TAKEN: {time.time() - start}')
        return
    for i in range(N):
        if (adj[curr_path[level-1]][i] != 0 and visited[i] == False):
            
            temp = curr_bound
            #if need reset
            
            curr_weight += adj[curr_path[level - 1]][i]
            #update chi phí
            if level == 1:
                curr_bound -= firstMin(adj, i)
            else:
                curr_bound -= ((secondMin(adj, curr_path[level - 1]) + firstMin(adj, i)) / 2)
                
            if curr_bound + curr_weight < final_res:
                curr_path[level] = i
                #có thể chọn i vào solution;
                visited[i] = True
                TSPRec(adj, curr_bound, curr_weight, level + 1, curr_path, visited)
                
            #nếu không thỏa mãn, reset;
            curr_weight -= adj[curr_path[level - 1]][i]
            curr_bound = temp
            visited = [False] * len(visited)
            for j in range(level):
                if curr_path[j] != -1:
                    visited[curr_path[j]] = True

In [ ]:
def TSP(adj):
    curr_bound = 0
    curr_path = [-1] * (N + 1)
    visited = [False] * N
    for i in range(N):
        curr_bound += (firstMin(adj, i) + secondMin(adj, i))
        #giới hạn dưới của cost
    curr_bound = math.ceil(curr_bound / 2)
    visited[0] = True
    curr_path[0] = 0
    TSPRec(adj, curr_bound, 0, 1, curr_path, visited)

In [ ]:
N = len(distance_matrix)
final_path = [None] * (N + 1)
visited = [False] * N
final_res = maxsize
start = time.time()
TSP(distance_matrix)
print("MIN_COST :", final_res)
print("PATH : ", end = ' ')
for i in range(N + 1):
    print(final_path[i], end = ' ')

<span style="font-size: 45px; bold: bolder">RESULT</span>

In [ ]:
dataset_name = [
    "5_TSP.txt", "15_TSP.txt", "17_TSP.txt", "26_TSP.txt",
    "42_TSP.txt", "48_TSP.txt", "29Y_TSP.txt", "51eli_TSP.txt",
    "52berlin_TSP.txt", "70st_TSP.txt", "76pr_TSP.txt", "101eli_TSP.txt",
    "120gr_TSP.txt", "130ch_TSP.txt", "150ch_TSP.txt"
]
optimal_solution = [
    19, 291, 208.5, 937, 699, 335.51,
    907.414, 429.98, 754.4365, 678.597,
    108.15943, 642.31, 166.6508, 611.086, 653.228
]
NN_solution = [
    21, 291, 217.8, 965, 864, 379.28,
    996.478, 505.77, 818.2191, 761.689, 130.921004,
    736.37, 185.026, 719.874, 707.844
]
ACO_solution = [
    19, 291, 217.8, 941, 768, 365.26, 
    931.862, 455.85, 766.359, 740.56, 123.77579,
    721.66, 181.683, 650.218, 679.947
]
GA_solution = [
    19, 291, 217.0, 998, 733, 360.76,
    907.792, 474.12, 787.036, 761.09, 120.1067,
    683.69, 177.839, 684.782, 774.485
]
length = len(dataset_name)
plt.figure(figsize=(8, 4))
plt.plot(dataset_name, optimal_solution, label='Optimal Solution', )
plt.plot(dataset_name, NN_solution, label='NN Solution')
plt.plot(dataset_name, ACO_solution, label='ACO Solution')
plt.plot(dataset_name, GA_solution, label='GA Solution')
plt.xticks(rotation=45)
plt.xlabel('Dataset_name')
# Đặt tên cho trục Oy
plt.ylabel('Value')
# Đặt tiêu đề cho đồ thị
plt.title('Different from optimal solution')
# Hiển thị legend
plt.legend()
# Hiển thị đồ thị
plt.tight_layout()
plt.show()

In [ ]:
for i in range(length):
    NN_solution[i] = (NN_solution[i] - optimal_solution[i]) / optimal_solution[i]
    ACO_solution[i] = (ACO_solution[i] - optimal_solution[i]) / optimal_solution[i]
    GA_solution[i] = (GA_solution[i] - optimal_solution[i]) / optimal_solution[i]
plt.figure(figsize=(8, 4))
plt.plot(dataset_name, NN_solution, label='NN Solution')
plt.plot(dataset_name, ACO_solution, label='ACO Solution')
plt.plot(dataset_name, GA_solution, label='GA Solution')
plt.xticks(rotation=45)
plt.xlabel('Dataset_name')
# Đặt tên cho trục Oy
plt.ylabel('Percent')
# Đặt tiêu đề cho đồ thị
plt.title('Different from optimal solution (%)')
# Hiển thị legend
plt.legend()
# Hiển thị đồ thị
plt.tight_layout()
plt.show()

In [ ]:
def drawMap(city, answer, title=""):
    for j in city:
        plt.plot(j[1], j[2], "ro")
        plt.annotate(j[0], (j[1], j[2]))
    for i in range(len(answer)):
        try:
            first = int(answer[i])
            secend = int(answer[i + 1])
            plt.plot([city[first-1][1], city[secend-1][1]], [city[first-1][2], city[secend-1][2]], "gray")
        except:
            continue
    first = int(answer[0])
    secend = int(answer[-1])
    plt.plot([city[first-1][1], city[secend-1][1]], [city[first-1][2], city[secend-1][2]], "gray")
    plt.title(title)
    plt.show()
    
fig = plt.figure(figsize=(10, 6))
drawMap(cities, optimal_road, f"Best solution")